> https://geckopy.readthedocs.io/en/latest/01-getting-started.html

> https://github.com/ginkgobioworks/geckopy (最新)

- geckopyはPyPIには登録されていない

In [1]:
import numpy as np
import pandas as pd
import cobra
from cobra.io import load_matlab_model
from cobra.io import write_sbml_model
from cobra.flux_analysis.variability import flux_variability_analysis
import geckopy
from geckopy.io import sbml

### Import models

In [2]:
# パス定義
yeast_ecgem_mat = r"/home/daiki/Github/ENEOS/python/models/ecYeastGEM_batch (CellFactory-ecYeastGEM).mat"
yeast_ecgem_xml = r"/home/daiki/Github/ENEOS/python/models/ecYeastGEM_batch (CellFactory-ecYeastGEM).xml"

In [ ]:
# matを読み込み
mat_model = load_matlab_model(yeast_ecgem_mat)

# SBMLで保存
write_sbml_model(mat_model, yeast_ecgem_xml)

In [3]:
# geckopyでSBMLを読み込み
ec_model = sbml.read_sbml_ec_model(yeast_ecgem_xml)

### Python API: Protein isolation

In [6]:
solution_rxn, solution_prot = ec_model.optimize() # cobrapyの solution オブジェクトを返す

# ec_model.slim_optimize() は目的関数の値（成長率など）を返すだけなので高速

In [7]:
solution_rxn # 各反応のフラックス (.fluxes)

,fluxes,reduced_costs
r_0006,20.622659,2.775558e-17
r_0070,0.000000,0.000000e+00
r_0094,0.000000,-9.653116e-02
r_0099,0.000000,-5.204170e-18
r_0200,0.000000,-6.938894e-18
...,...,...
r_4484_REVNo1,0.000000,-1.387779e-17
r_4484_REVNo2,0.000000,-2.037970e-17
r_4484_REVNo3,0.000000,-2.037970e-17
draw_prot_A2P2R3,0.000000,0.000000e+00


In [8]:
solution_prot # タンパク質濃度とタンパク質影響度 (.contribution)(双対変数)

,fluxes,reduced_costs
prot_P00045,2.100937e-19,0.000000e+00
prot_P32891,0.000000e+00,-0.000000e+00
prot_P00044,0.000000e+00,-0.000000e+00
prot_P39976,0.000000e+00,-0.000000e+00
prot_P46681,0.000000e+00,-0.000000e+00
...,...,...
prot_Q04119,0.000000e+00,-8.090059e-13
prot_P38995,0.000000e+00,-0.000000e+00
prot_P49954,0.000000e+00,0.000000e+00
prot_P39979,0.000000e+00,-0.000000e+00


### Kcats

- geckopy では、各酵素-反応ペアに対応する触媒定数 ($k_{cat}$) を `Protein`オブジェクト経由で **辞書型 (dict)** として扱う。
- $k_{cat}$ の入力単位はユーザー側が指定するのは $1/s$ $(秒^{-1})$ だが、内部では $1/h$ $(時間^{-1})$ に変換されている。
$$
k_{cat} [s^{-1}] \times 3600 \to k_{cat} [h^{-1}]
$$

In [24]:
# kcat の参照と操作
print(ec_model.proteins.prot_P00045.kcats) # Proteinオブジェクト(prot_P00045)のkcatを確認
print(ec_model.proteins.prot_P00045.kcats["r_0004No1"]) # "r_0004No1" に対応する kcat

ec_model.proteins.prot_P00045.kcats["r_0004No1"] = 100 # 新たに kcat を設定

{<Reaction r_0437No1 at 0x7f2fb210ba30>: 999.9920000639995, <Reaction r_0002No1 at 0x7f2fbfbf56a0>: 1499.9610010139734, <Reaction r_0004No1 at 0x7f2fbfb7b370>: 120.0, <Reaction r_0001No1 at 0x7f2fbfd1ad90>: 1499.9610010139734, <Reaction r_0001No3 at 0x7f2fbfc2bbe0>: 1499.9610010139734}
120.0


In [25]:
# 化学量論行列 S の量論係数 s を取得
ec_model.reactions.r_0004No1.metabolites[ec_model.proteins.prot_P00045] # - 1/kcat (実際は - 1/(kcat*3600) の値が出力される) 

-2.777777777777778e-06

In [26]:
ec_model.proteins.prot_P00045.kcats["r_0004No1"] = 120 # 元の値が60に対して2倍の kcat を設定すれば、s は 1/2倍される
ec_model.reactions.r_0004No1.metabolites[ec_model.proteins.prot_P00045]

-2.3148148148148148e-06

## Thermodynamics integretion

- geckopyは、pytfaを通してTFA (thermodynamics flux analysis) と enzyme constraint model を統合できる。

### Load the model

### Plain FVA

In [12]:
ecM = ec_model.copy()
ecM.solver = 'glpk'
print(ecM.objective)

ecM.objective = "r_4041" # biomass reaction
print(ecM.objective)

Maximize
1.0*r_2111 - 1.0*r_2111_reverse_58b69
Maximize
1.0*r_4041 - 1.0*r_4041_reverse_2a391


In [ ]:
for rxn in ecM.exchanges:
    rxn.bounds = 0.0, 0.0
ecM.reactions.r_1714.bounds = -10.0, 0.0 # glucose exchange
ecM.reactions.r_1718.bounds = -10.0, 0.0 # xylose exchange
ecM.reactions.r_1093No1 = 0.0, 1000.0 # xylose reductase
ecM.reactions.r_2104 = 0.0, 0.0 # xylitol exchange
ecM.reactions.r_1092No1 = 0.0, 1000.0 # xylitol dehydrogenase
ecM.reactions.r_4490 = 0.0, 1000.0 # xylulose⇒xylitol
ecM.reactions.r_1654 = -1000.0, 0.0 # ammonium exchange
ecM.reactions.r_1992 = -10.0, 0.0 # oxygen exchange
ecM.reactions.r_1861 = -1000.0, 0.0 # iron(2+) exchange
ecM.reactions.r_2005 = -1000.0, 0.0 # phosphate exchange
ecM.reactions.r_2020 = -1000.0, 0.0 # potassium exchange
ecM.reactions.r_2049 = -1000.0, 0.0 # sodium exchange
ecM.reactions.r_2060 = -1000.0, 0.0 # sulphate exchange
ecM.reactions.r_2100 = -1000.0, 0.0 # water exchange
ecM.reactions.r_4593 = -1000.0, 0.0 # chloride change
ecM.reactions.r_4594 = -1000.0, 0.0 # Cu2(+) exchange
ecM.reactions.r_4596 = -1000.0, 0.0 # Zn(2+) exchange
ecM.reactions.r_4595 = -1000.0, 0.0 # Mn(2+) exchange
ecM.reactions.r_4597 = -1000.0, 0.0 # Mg(2+) exchange
ecM.reactions.r_4600 = -1000.0, 0.0 # Ca(2+) exchange

In [18]:
ecM.optimize()

/home/daiki/Github/ENEOS/python/GECKO/.venv/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


(<Solution infeasible at 0x7fb1667d7ca0>,
 <Solution infeasible at 0x7fb1667d7d90>)

In [6]:
fva_fluxes = flux_variability_analysis(
    model = ecM, 
    reaction_list = None,
    loopless = False, 
    fraction_of_optimum = 0.9, 
    pfba_factor = 1.1,
    processes = 4
)

Infeasible: There is no optimal solution for the chosen objective! (infeasible).

In [28]:
import plotly.express as px
dfs = (fva_fluxes)
df_plot = pd.concat([abs(dfs.maximum - dfs.minimum).apply(lambda x: x if x < 2000 else 2000) for df in dfs], axis=1).melt(var_name="Variability method", value_name="Flux")
df_plot.loc[df_plot["Variability method"] == 0, "Variability method"] = "FBA"
fig = px.histogram(
    df_plot,
    x="Flux", color="Variability method",
    cumulative=True, nbins=200,
    color_discrete_sequence=["rgba(26, 200, 26, 0.8)", "rgba(200, 26, 80, 0.5)", "rgba(26, 26, 200, 0.5)"],
    marginal="violin", barmode="overlay",
)
fig.show(renderer="notebook_connected")